In [12]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.optimize import least_squares
from sklearn.linear_model import LinearRegression
from python_module.pricing_model import SABRModel, BlackScholesModel

In [13]:
F = 200
T = 1
alpha = 0.1
rho = -0.25
nu = 0.7
n_steps = 20
n_paths = 100000
spot, vol = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=1, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths)
market_mean = spot.to_numpy().flatten().mean()
market_std = spot.to_numpy().flatten().std()
market_skew = stats.skew(spot.to_numpy().flatten())
market_kurtosis = stats.kurtosis(spot.to_numpy().flatten())

In [14]:
def objective_function(params, F, T, n_steps, n_paths, market_mean, market_std, market_skew, market_kurtosis):

    alpha, rho, nu = params
    spot, vol = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=1, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths)
    model_mean = spot.to_numpy().flatten().mean()
    model_std = spot.to_numpy().flatten().std()
    model_skew = stats.skew(spot.to_numpy().flatten())
    model_kurtosis = stats.kurtosis(spot.to_numpy().flatten())
    errors = np.array([market_mean, market_std, market_skew, market_kurtosis]) - np.array([model_mean, model_std, model_skew, model_kurtosis])
    return errors

init_guess = [0.1, 0.0, 0.3]
bounds = ([1e-6, -0.9999, 1e-6], [1, 0.9999, 3])
result = least_squares(objective_function, x0=init_guess, args=(F, T, n_steps, n_paths, market_mean, market_std, market_skew, market_kurtosis), bounds=bounds)
calibrated_alpha, calibrated_rho, calibrated_nu = result.x
calibrated_alpha, calibrated_rho, calibrated_nu

(0.09999999999998323, -0.24999999999701775, 0.6999999999997523)

In [15]:
def solve_realized_parameters(F, spot_timeseries, T, n_paths=100000):
    n_steps = len(spot_timeseries)
    market_mean = spot_timeseries.mean()
    market_std = spot_timeseries.std()
    market_skew = stats.skew(spot_timeseries)
    market_kurtosis = stats.kurtosis(spot_timeseries)

    def objective_function(params, F, T, n_steps, n_paths, market_mean, market_std, market_skew, market_kurtosis):

        alpha, rho, nu = params
        spot, vol = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=1, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths)
        model_mean = spot.to_numpy().flatten().mean()
        model_std = spot.to_numpy().flatten().std()
        model_skew = stats.skew(spot.to_numpy().flatten())
        model_kurtosis = stats.kurtosis(spot.to_numpy().flatten())
        errors = np.array([market_mean, market_std, market_skew, market_kurtosis]) - np.array([model_mean, model_std, model_skew, model_kurtosis])
        return errors

    init_guess = [0.1, 0.0, 0.3]
    bounds = ([1e-6, -0.9999, 1e-6], [1, 0.9999, 3])
    result = least_squares(objective_function, x0=init_guess, args=(F, T, n_steps, n_paths, market_mean, market_std, market_skew, market_kurtosis), bounds=bounds)
    calibrated_alpha, calibrated_rho, calibrated_nu = result.x
    return calibrated_alpha, calibrated_rho, calibrated_nu